In [1]:
import sys
print(sys.version)

3.8.5 (default, Sep  3 2020, 21:29:08) [MSC v.1916 64 bit (AMD64)]


In [12]:
!pip install MulticoreTSNE

  Using cached MulticoreTSNE-0.1.tar.gz (20 kB)
  Running setup.py clean for MulticoreTSNE
Failed to build MulticoreTSNE
    Running setup.py install for MulticoreTSNE: started
    Running setup.py install for MulticoreTSNE: finished with status 'error'


  ERROR: Command errored out with exit status 1:
   command: 'C:\ProgramData\Anaconda3\python.exe' -u -c 'import sys, setuptools, tokenize; sys.argv[0] = '"'"'C:\\Users\\Jessie Houng\\AppData\\Local\\Temp\\pip-install-3mekfcpr\\multicoretsne\\setup.py'"'"'; __file__='"'"'C:\\Users\\Jessie Houng\\AppData\\Local\\Temp\\pip-install-3mekfcpr\\multicoretsne\\setup.py'"'"';f=getattr(tokenize, '"'"'open'"'"', open)(__file__);code=f.read().replace('"'"'\r\n'"'"', '"'"'\n'"'"');f.close();exec(compile(code, __file__, '"'"'exec'"'"'))' bdist_wheel -d 'C:\Users\Jessie Houng\AppData\Local\Temp\pip-wheel-ql2ql76o'
       cwd: C:\Users\Jessie Houng\AppData\Local\Temp\pip-install-3mekfcpr\multicoretsne\
  Complete output (46 lines):
  running bdist_wheel
  running build
  running build_py
  creating build
  creating build\lib.win-amd64-3.8
  creating build\lib.win-amd64-3.8\MulticoreTSNE
  copying MulticoreTSNE\__init__.py -> build\lib.win-amd64-3.8\MulticoreTSNE
  creating build\lib.win-amd64-3.8\Mul

In [10]:
import os
import sklearn.manifold
import matplotlib.pyplot as plt
import h5py
import chart_studio.plotly.plotly as py
import plotly.graph_objs as go
import plotly.offline as offline
import numpy as np
import collections
import pandas as pd
import itertools
import seaborn as sns
import time
import json
import re
from MulticoreTSNE import MulticoreTSNE as TSNE
# %load_ext wurlitzer

offline.init_notebook_mode()


flatten = lambda l: [item for sublist in l for item in sublist]

def load_embeddings(path):
    f = h5py.File(path, 'r')
    nemb = f['nemb'][:]
    f.close()
    return nemb[1:]


def load_vocab(path):
    vocab = []
    with open(path, 'r') as f:
        for line in f.readlines():
            split = line.split(' ')
            vocab.append((split[0].replace('\'', ''), int(split[1].rstrip())))
    # ignore UNK at position 0
    return vocab[1:]


def run_tsne(nemb, multicore=True):
    if multicore:
        tsne = TSNE(n_jobs=4)
    else:
        tsne = sklearn.manifold.TSNE(perplexity=30, n_components=2, init='pca', n_iter=5000, verbose=1)
    return tsne.fit_transform(nemb)


def load_recipes(path):
    recipes = []
    with open(path, 'r') as f:
        for line in f:
            if line[0] == '#':
                pass
            else:
                recipes.append(line.rstrip().split(','))
    return recipes
   
    
def build_food2cuisine(recipes, vocab):
    foods = [tup[0] for tup in vocab]
    food_counters = {food: collections.Counter() for food in foods}
    cuisine_counter = collections.Counter()
    for line in recipes:
            cuisine = line[0]
            cuisine_counter.update([cuisine])
            for food in line[1:]:
                if food in foods:
                    food_counters[food].update([cuisine])
    food2cuisine = {}
    for food, food_counter in food_counters.items():
        for cuisine in cuisine_counter.keys():
            food_counter[cuisine] = np.float32(food_counter[cuisine]) / np.float32(cuisine_counter[cuisine])
        sorted_food_counter = sorted(food_counter.items(), key=lambda a: a[1])[::-1]
        print(food, sorted_food_counter[0:2])
        food2cuisine.update({food: sorted_food_counter[0][0]})
    return food2cuisine


# These are the "Tableau 20" colors as RGB.    
tableau20 = [(31, 119, 180), (174, 199, 232), (255, 127, 14), (255, 187, 120),    
             (44, 160, 44), (152, 223, 138), (214, 39, 40), (255, 152, 150),    
             (148, 103, 189), (197, 176, 213), (140, 86, 75), (196, 156, 148),    
             (227, 119, 194), (247, 182, 210), (127, 127, 127), (199, 199, 199),    
             (188, 189, 34), (219, 219, 141), (23, 190, 207), (158, 218, 229)]    
tableau20_rgb = ['rgb' + str(triplet) for triplet in tableau20]

# Prettify ingredients
pretty_food = lambda s: ' '.join(s.split('_')).capitalize().lstrip()
# Prettify cuisine names
pretty_cuisine = lambda s: ''.join(map(lambda x: x if x.islower() else " "+x, s)).lstrip()


def make_plot(name, points, labels, legend_labels, legend_order, legend_label_to_color, pretty_legend_label, publish):
    lst = zip(points, labels, legend_labels)
    full = sorted(lst, key=lambda x: x[2])
    traces = []
    for legend_label, group in itertools.groupby(full, lambda x: x[2]):
        group_points = []
        group_labels = []
        for tup in group:
            point, label, _ = tup
            group_points.append(point)
            group_labels.append(label)
        group_points = np.stack(group_points)
        traces.append(go.Scattergl(
            x = group_points[:, 0],
            y = group_points[:, 1],
            mode = 'markers',
            marker = dict(
                color = legend_label_to_color[legend_label],
                size = 8,
                opacity = 0.6,
                #line = dict(width = 1)
            ),
            text = ['{} ({})'.format(label, pretty_legend_label(legend_label)) for label in group_labels],
            hoverinfo = 'text',
            name = legend_label
        )
        )
    # order the legend
    ordered = [[trace for trace in traces if trace.name == lab] for lab in legend_order]
    traces_ordered = flatten(ordered)
    def _set_name(trace):
        trace.name = pretty_legend_label(trace.name)
        return trace
    traces_ordered = list(map(_set_name, traces_ordered))
    layout = go.Layout(
        xaxis=dict(
            autorange=True,
            showgrid=False,
            zeroline=False,
            showline=False,
            dtick=True,
            ticks='',
            showticklabels=False
        ),
        yaxis=dict(
            autorange=True,
            showgrid=False,
            zeroline=False,
            showline=False,
            dtick=True,
            ticks='',
            showticklabels=False
        )
    )
    fig = go.Figure(data=traces_ordered, layout=layout)
    if publish:
        plotter = py.iplot
    else:
        plotter = offline.plot
    plotter(fig, filename=name + '.html')

ModuleNotFoundError: No module named 'MulticoreTSNE'

In [33]:
# path = '/home/jaan/fit/food2vec/2017-01-24'
path = '../fit/'
nemb = load_embeddings(os.path.join(path, 'embeddings.h5'))
vocab = load_vocab(os.path.join(path, 'vocab.txt'))
food2id = {tup[0]: i for i, tup in enumerate(vocab)}

## Plot ingredients

In [34]:
# don't plot UNK at position 0
low_dim_embs = run_tsne(nemb.astype(np.float64), multicore=False)

[t-SNE] Computing 91 nearest neighbors...
[t-SNE] Indexed 2087 samples in 0.000s...
[t-SNE] Computed neighbors for 2087 samples in 0.091s...
[t-SNE] Computed conditional probabilities for sample 1000 / 2087
[t-SNE] Computed conditional probabilities for sample 2000 / 2087
[t-SNE] Computed conditional probabilities for sample 2087 / 2087
[t-SNE] Mean sigma: 0.281192
[t-SNE] KL divergence after 250 iterations with early exaggeration: 75.839890
[t-SNE] KL divergence after 5000 iterations: 1.471477


In [35]:
recipes = load_recipes('../dat/all-recipes.csv')

In [36]:
food2cuisine = build_food2cuisine(recipes, vocab)

garlic [('SoutheastAsian', 0.64025754), ('LatinAmerican', 0.5451564)]
pepper [('LatinAmerican', 0.6219307), ('African', 0.54049444)]
butter [('EasternEuropean', 0.52755904), ('NorthernEuropean', 0.5060893)]
onion [('EasternEuropean', 0.42519686), ('LatinAmerican', 0.38698286)]
egg [('EasternEuropean', 0.51706034), ('NorthAmerican', 0.37170342)]
wheat [('EasternEuropean', 0.51706034), ('MiddleEastern', 0.42170542)]
salt [('SouthAsian', 0.5345495), ('NorthernEuropean', 0.38971582)]
milk [('NorthAmerican', 0.26376984), ('EasternEuropean', 0.24671917)]
chicken [('SoutheastAsian', 0.38633817), ('African', 0.36999148)]
cream [('NorthernEuropean', 0.34100136), ('EasternEuropean', 0.2913386)]
cheese [('SouthernEuropean', 0.48942024), ('LatinAmerican', 0.29112008)]
olive [('African', 0.49872124), ('SouthernEuropean', 0.44406828)]
tomato [('LatinAmerican', 0.19483687), ('SouthernEuropean', 0.18528706)]
vanilla [('NorthAmerican', 0.19324651), ('EasternEuropean', 0.14698163)]
vegetable_oil [('East

In [37]:
with open('../dat/food2cuisine.json', 'w') as f:
    json.dump(food2cuisine, f, indent=2)

In [38]:
cuisines = list(set(food2cuisine.values()))
# np.random.seed(1234)
# tableau20_sample = np.random.choice(tableau20_rgb, len(cuisines), replace=False)
# cuisine2color = {cuisine: tableau20_sample[i] for i, cuisine in enumerate(cuisines)}
cuisine2color = {
    'African': sns.xkcd_rgb["grey"],
    'LatinAmerican': sns.xkcd_rgb["forest green"],
    'NorthAmerican': sns.xkcd_rgb["light pink"],
    'MiddleEastern': sns.xkcd_rgb["mustard yellow"],
    'EastAsian': sns.xkcd_rgb["orange"],
    'SouthAsian': sns.xkcd_rgb["magenta"],
    'SoutheastAsian': sns.xkcd_rgb["purple"],
    'NorthernEuropean': sns.xkcd_rgb["blue"],
    'EasternEuropean': sns.xkcd_rgb["deep blue"],
    'WesternEuropean': sns.xkcd_rgb["sky blue"],
    'SouthernEuropean': sns.xkcd_rgb["olive"],
}
food2color = {food: cuisine2color[food2cuisine[food]] for food in food2cuisine.keys()}

In [39]:
legend_order = [
'African',
'LatinAmerican',
'NorthAmerican',
'EastAsian',
'SouthAsian',
'SoutheastAsian',
'MiddleEastern',
'NorthernEuropean',
'EasternEuropean',
'WesternEuropean',
'SouthernEuropean',
]

In [104]:
labels = [item[0] for item in vocab]
legend_labels = [food2cuisine[food] for food in labels]
labels = [item[0] for item in vocab]
labels = map(pretty_food, labels)
# legend_order = cuisine2color.keys()
make_plot(name='food2vec_food_embeddings_tsne',
          points=low_dim_embs, 
          labels=labels, 
          legend_labels=legend_labels, 
          legend_order=legend_order, 
          legend_label_to_color=cuisine2color, 
          pretty_legend_label=pretty_cuisine,
          publish=False)

In [109]:
labels = [item[0] for item in vocab]
labels[0]
pretty_food(labels[0])
print(low_dim_embs[0], pretty_food(labels[0]))

[-32.0945    15.967439] Garlic


In [114]:
labels.index("crisps")

1651

In [115]:
low_dim_embs[1651]

array([-3.450662e-02,  6.261346e+01], dtype=float32)

In [89]:
print(tempLabels)

{}


In [41]:
len(vocab)

2087

## Plot recipes
NB: TSNE Takes ~10-30 minutes on 50k recipes

In [42]:
def build_recipe_embedding(recipes, nemb, food2id):
    """Get the recipe embedding.
    
    A recipe's embedding is the mean of its ingredients' embeddings.
    
    Args:
        recipes: list of recipes in the form [cuisine, food1, food2, ...]
        nemb: normalized embeddings
        food2id: map from food string to index in normalized embeddings
    Returns:
        List of tuples, each tuple has form (cuisine, ingredients, recipe embedding)
    """
    recipe_embeddings = []
    for line in recipes:
        cuisine = line.pop(0)
        foods = line
        # check that we have learned the embeddings for all the ingredients
        filtered_foods = [food for food in foods if food in food2id]
        if len(filtered_foods) > 0:
            food_ids = list(map(lambda x: food2id[x], filtered_foods))
            embedding = np.mean(nemb[food_ids], axis=0)
            recipe_embeddings.append((cuisine, foods, embedding))
    return recipe_embeddings

In [43]:
recipe_embeddings = build_recipe_embedding(recipes, nemb, food2id)

In [118]:
print(recipe_embeddings[0:5])

[('African', ['chicken', 'cinnamon', 'soy_sauce', 'onion', 'ginger'], array([ 0.05470821, -0.0243897 ,  0.06145782, -0.02935698, -0.04946625,
        0.08577897, -0.01597825, -0.10209765, -0.02926719,  0.05147906,
       -0.10030527,  0.11119102,  0.0436901 , -0.13690129,  0.01589183,
        0.04792665,  0.17198846, -0.07892211, -0.02907386,  0.02380269,
        0.01535639, -0.06844411, -0.03833026,  0.00619347,  0.02865616,
       -0.01161167,  0.04109917, -0.04929319,  0.06221039, -0.14173038,
       -0.08577577,  0.13345847, -0.00423009,  0.03494855,  0.03590785,
        0.01842419, -0.02325651, -0.0726238 ,  0.08373059,  0.10517545,
       -0.03869651, -0.05869976, -0.02818147,  0.02767706,  0.06101222,
       -0.03775068,  0.0222307 , -0.01908464,  0.0425612 , -0.09481445,
       -0.01820208, -0.01275918, -0.06671564, -0.01126305, -0.06638577,
        0.03092619,  0.13777794,  0.07108434, -0.09160206,  0.05076593,
        0.01243511, -0.02959444,  0.05492051,  0.08133496, -0.0713

In [44]:
# subset = np.random.choice(range(len(recipe_embeddings)), 2000, replace=False)
# small = [recipe_embeddings[idx] for idx in subset]

In [45]:
cuisine_labels, ingredients, embeddings = zip(*recipe_embeddings)
cuisine_labels = list(cuisine_labels)
recipe_nemb = np.vstack(embeddings)

In [46]:
cuisine_counter = collections.Counter(cuisine_labels)

In [47]:
cuisine_counter

Counter({'African': 1173,
         'EastAsian': 7435,
         'EasternEuropean': 381,
         'LatinAmerican': 11892,
         'MiddleEastern': 645,
         'NorthAmerican': 45842,
         'NorthernEuropean': 739,
         'SouthAsian': 3618,
         'SoutheastAsian': 3572,
         'SouthernEuropean': 14178,
         'WesternEuropean': 6774})

In [48]:
recipe_emb_path = os.path.join(path, 'low_dim_recipe_embs.npz')

In [49]:
%load_ext wurlitzer
t0 = time.time()
low_dim_recipe_embs = run_tsne(recipe_nemb.astype(np.float64))
np.savez_compressed(recipe_emb_path, low_dim_recipe_embs)
print('time to run tsne on %d points: %.3f mins' % (len(recipe_nemb), (time.time() - t0) / 60.))

time to run tsne on 96249 points: 4.960 mins


In [50]:
with open(recipe_emb_path, 'rb') as f:
    low_dim_recipe_embs = np.load(f)['arr_0']

In [51]:
# low_dim_recipe_embs = run_fast_tsne(embeddings)
# low_dim_recipe_embs = tsne.bh_sne(embeddings)
# t0 = time.time()
# low_dim_recipe_embs = bhtsne.run_bh_tsne(nemb, no_dims=2, perplexity=50, theta=0.5, randseed=-1, verbose=False,initial_dims=50, use_pca=True, max_iter=1000)
# print 'time to run tsne on %d points: %.3f mins' % (len(recipe_nemb), (time.time() - t0) / 60.)
low_dim_recipe_embs_list = low_dim_recipe_embs.tolist()

In [52]:
# clean_string = lambda x: re.sub(r'([^\s\w]|_)+', '', x)
recipe_labels = [', '.join([pretty_food(food) for food in foods]).lower().capitalize() for foods in ingredients]

In [53]:
make_plot(name='food2vec_recipe_embeddings_tsne',
          points=low_dim_recipe_embs_list, 
          labels=recipe_labels, 
          legend_labels=cuisine_labels, 
          legend_order=legend_order, 
          legend_label_to_color=cuisine2color, 
          pretty_legend_label=pretty_cuisine,
          publish=False)

## Cuisine embeddings

In [54]:
# cuisine embedding as the average of recipe embeddings:
recipe_embeddings[0]
sorted_recipe_embeddings = sorted(recipe_embeddings, key=lambda x: x[0])
cuisine_embeddings = []
for cuisine_name, group in itertools.groupby(sorted_recipe_embeddings, lambda x: x[0]):
    cuisine_recipe_emb = []
    for tup in group:
        _, _, recipe_emb = tup
        cuisine_recipe_emb.append(recipe_emb)
    all_cuisine_recipe_emb = np.stack(cuisine_recipe_emb)
    cuisine_emb = np.mean(all_cuisine_recipe_emb, axis=0)
    cuisine_embeddings.append((cuisine_name, cuisine_emb))  


# cuisine embedding as the average of food embeddings with highest relative prevalence in that cuisine
# def reverse_dict(mydict):
#     reversed_dict = collections.defaultdict(list)
#     for key,value in mydict.iteritems():
#         reversed_dict[value].append(key)
#     return reversed_dict
# cuisine2foods = reverse_dict(food2cuisine)
# cuisine_embeddings = []
# for cuisine, foods in cuisine2foods.items():
#     food_ids = [food2id[food] for food in foods]
#     food_embs = nemb[food_ids]
#     cuisine_embeddings.append((cuisine, np.mean(food_embs, axis=0)))

In [55]:
t0 = time.time()
cuisine_names, cuisine_emb = zip(*cuisine_embeddings)
cuisine_emb = np.asarray(cuisine_emb)
low_dim_cuisine_embs = run_tsne(cuisine_emb, multicore=False)
print('time to run tsne on %d points: %.3f mins' % (len(cuisine_emb), (time.time() - t0) / 60.))

[t-SNE] Computing 10 nearest neighbors...
[t-SNE] Indexed 11 samples in 0.000s...
[t-SNE] Computed neighbors for 11 samples in 0.001s...
[t-SNE] Computed conditional probabilities for sample 11 / 11
[t-SNE] Mean sigma: 1125899906842624.000000
[t-SNE] KL divergence after 250 iterations with early exaggeration: 45.037586
[t-SNE] KL divergence after 1300 iterations: 0.268368
time to run tsne on 11 points: 0.021 mins


In [64]:
make_plot(name='food2vec_cuisine_embeddings_tsne',
          points=low_dim_cuisine_embs, 
          labels=cuisine_names, 
          legend_labels=cuisine_names, 
          legend_order=legend_order, 
          legend_label_to_color=cuisine2color, 
          pretty_legend_label=pretty_cuisine,
          publish=False)

## Write foods to json

In [57]:
foods = [tup[0] for tup in vocab]
food2prettyfood = [{"value": food, "text": pretty_food(food)} for food in foods]
food2prettyfood.append([{"value": tup[0], "text": pretty_cuisine(tup[0])} for tup in cuisine_embeddings])
with open(os.path.join(path, 'foods.json'), 'w') as f:
    json.dump(food2prettyfood, f, indent=4)

In [62]:
def write_to_js(words, embeddings, path):
    word_vecs = {}
    for word, embedding in zip(words, embeddings):
        word_vecs[word] = embedding.tolist()
    with open(path, 'w') as f:
        f.write('var wordVecs=')
        json.dump(word_vecs, f)
        f.write(';')
# lower precision, faster
# nemb = nemb.astype(np.float16)
words = [pretty_food(food) for food in foods] + [pretty_cuisine(tup[0]) for tup in cuisine_embeddings]
all_emb = np.vstack([nemb, cuisine_emb])
# '../../word2vecjson/data/foodVecs.js'
write_to_js(words, all_emb, path=os.path.join(path, 'foodVecs.js'))
display(path=os.path.join(path, 'foodVecs.js'))

In [63]:
# print list of foods for autocomplete in assets/js/initm.js
string = str({word: None for word in words})
with open(os.path.join(path, 'javascript_dict.txt'), 'w') as f:
    f.write(string.replace('None', 'null'))

'/home/ax/Downloads/temp/foodTemp/food2vec/src'